# 1장 멀티패러다임이 현대 언어를 확장하는 방법

## 1.1 객체지향 디자인 패턴의 반복자 패턴과 일급 함수

### 1.1.1 GoF의 반복자 패턴

컬렉션의 내부 구조를 노출하는 대신 next() 같은 public 메서드로 내부 요소에 접근할 수 있도록 설계 -> 컬렉션의 실제 구조와 상관없이 일관된 방식으로 순회할 수 있게 한다.

#### 반복자 패턴의 Iterator 인터페이스 예시


In [ ]:
interface IteratorYieldResult<T> {
  done?: false;
  value: T;
}

interface IteratorReturnResult {
  done: true;
  value: undefined;
}

interface Iterator<T> {
  next(): IteratorYieldResult<T> | IteratorReturnResult;
}

### 1.1.2 ArrayLike로부터 Iterator 생성하기

#### Class 형태로 구현해보는 ArrayLike Iterator

In [ ]:
/* lib.es5.ts
interface ArrayLike<T> {
  readonly length: number;
  readonly [n: number]: T;
} 
*/

class ArrayLikeIterator<T> implements Iterator<T> {
  private index = 0;
  constructor(private arrayLike: ArrayLike<T>) {}

  next(): IteratorResult<T> {
    if (this.index < this.arrayLike.length) {
      return {
        value: this.arrayLike[this.index++],
        done: false,
      };
    } else {
      return {
        value: undefined,
        done: true,
      }
    }
  }
}

const arrayLike: ArrayLike<number> = {
  0: 10,
  1: 20,
  2: 30,
  length: 3
};

const iterator: Iterator<number> = new ArrayLikeIterator(arrayLike);

console.log(iterator.next()); // { value: 10, done: false }
console.log(iterator.next()); // { value: 20, done: false }
console.log(iterator.next()); // { value: 30, done: false }
console.log(iterator.next()); // { value: undefined, done: true }

const array: Array<string> = ["a", "b", "c"];
const iterator2: Iterator<string> = new ArrayLikeIterator(array);

console.log(iterator2.next()); // { value: "a", done: false }
console.log(iterator2.next()); // { value: "b", done: false }
console.log(iterator2.next()); // { value: "c", done: false }
console.log(iterator2.next()); // { value: undefined, done: true }

{ value: 10, done: false }
{ value: 20, done: false }
{ value: 30, done: false }
{ value: undefined, done: true }
{ value: "a", done: false }
{ value: "b", done: false }
{ value: "c", done: false }
{ value: undefined, done: true }


: 

### 1.1.3 ArrayLike를 역순으로 순회하는 이터레이터 만들기

`array.reverse()` 메서드를 사용한 후에 역순으로 순회할 수도 있지만, 성능이 중요한 경우 이터레이터를 활용하면 지연 평가를 통해 불필요한 연산과 메모리 사용량을 줄이면서 역순으로 순회할 수 있다.

#### 이터레이터의 지연성을 이용한 reverse 함수 만들기

In [2]:
function reverse<T>(arrayLike: ArrayLike<T>): Iterator<T> {
  let idx = arrayLike.length;
  return {
    next() {
      if (idx === 0) {
        return { value: undefined, done: true };
      } else {
        return { value: arrayLike[--idx], done: false };
      }
    }
  }
};

const array = ['A', 'B'];
const reversed = reverse(array);
console.log(array); // ['A', 'B'] (원본 배열은 그대로)

console.log(reversed.next().value, reversed.next().value); // B A

[ "A", "B" ]
B A


#### 역순 순회 이터레이터의 장점 (vs 역순 정렬을 활용한 역순 순회)

- 역순 정렬 비용이 발생하지 않는다.
- 전체 순회를 하지 않을 경우, 연산과 메모리 사용을 최소화할 수 있다.
- 원본 배열 보존이 필요한 경우, 원본 배열을 복사하지 않아도 보존할 수 있다.

### 1.1.4 지연 평가되는 map 함수

In [ ]:
function map<A, B>(transform: (value: A) => B, iterator: Iterator<A>): Iterator<B> {
  return {
    next(): IteratorResult<B> {
      const { value, done } = iterator.next();
      return done
        ? { value, done }
        : { value: transform(value), done };
    }
  };
}

const array = ['A', 'B', 'C', 'D', 'E', 'F'];
const iterator = map(str => str.toLowerCase(), reverse(array));
console.log(iterator.next().value, iterator.next().value);  // f e

f e


: 

### 1.1.5 멀티패러다임의 교차점: 반복자 패턴과 일급 함수

**반복자 패턴**은 지연 평가가 가능한 객체를 생성할 수 있게 해주고 **일급 함수**는 고차 함수를 정의할 수 있게 한다.
두 가지를 조합하면 map, filter, take, reduce 등 지연 평가를 활용하거나 지연 평가된 리스트를 다루는 함수를 구현할 수 있다.

## 1.2 명령형 프로그래밍으로 이터레이터를 만드는 제너레이터 함수

### 1.2.1 제너레이터 기본 문법

#### 자연수의 무한 시퀀스를 생성하는 제너레이터 함수

In [ ]:
function* naturals() {
  let n = 1;
  while (true) {
    yield n++;
  }
}

const iter = naturals();

console.log(iter.next()); // { value: 1, done: false }
console.log(iter.next()); // { value: 2, done: false }
console.log(iter.next()); // { value: 3, done: false }

{ value: 1, done: false }
{ value: undefined, done: true }
{ value: undefined, done: true }


: 

### 1.2.2 제너레이터로 작성한 reverse 함수

In [ ]:
function* reverse<T>(arrayLike: ArrayLike<T>): IterableIterator<T> {
  let idx = arrayLike.length;
  while (idx) {
    yield arrayLike[--idx];
  }
}

const array = ['A', 'B', 'C', 'D', 'E', 'F'];
const reversed = reverse(array);

console.log(reversed.next().value); // F
console.log(reversed.next().value); // E
console.log(reversed.next().value); // D

F
E
D


: 

## 1.3 자바스크립트에서 반복자 패턴 사례: 이터레이션 프로토콜

### 1.3.1 이터레이터와 이터러블

#### 이터레이터

In [1]:
function naturals(end = Infinity): Iterator<number> {
  let n = 1;
  return {
    next(): IteratorResult<number> {
      return n <= end
        ? { value: n++, done: false }
        : { value: undefined, done: true };
    }
  }
}

const interator = naturals(3);

console.log(interator.next().value);  // 1
console.log(interator.next().value);  // 2
console.log(interator.next().value);  // 3
console.log(interator.next().done);   // true

1
2
3
true


#### for....of문으로 순회하려면

`[Symbol.iterator]() { return { next() { ... }};}` 메서드를 추가하여 이터러블 객체를 만들 수 있다.

In [3]:
function naturals(end = Infinity): Iterator<number> {
  let n = 1;
  return {
    next(): IteratorResult<number> {
      return n <= end
        ? { value: n++, done: false }
        : { value: undefined, done: true };
    },
    [Symbol.iterator]() {
      return this;
    }
  }
}

const iterator2 = naturals(3);

for (const num of iterator2) {
  console.log(num);
}

1
2
3


#### TypeScript의 `Iterator<T>`, `Iterable<T>`, `IterableIterator<T>` 인터페이스 정의의 핵심 부분

In [ ]:
interface IteratorYieldResult<T> {
  done?: false;
  value: T;
}

interface IteratorReturnResult {
  done: true;
  value: undefined;
}

interface Iterator<T> {
  next(): IteratorYieldResult<T> | IteratorReturnResult;
}

interface Iterable<T> {
  [Symbol.iterator](): Iterator<T>;
}

interface IterableIterator<T> extends Iterator<T> {
  [Symbol.iterator](): IterableIterator<T>;
}

#### 내장 이터러블

Array, Set, Map 등도 모두 이터러블이다.

In [ ]:
const array = [1, 2, 3];
const arrayIterator = array[Symbol.iterator]();

console.log(arrayIterator.next()); // { value: 1, done: false }
console.log(arrayIterator.next()); // { value: 2, done: false }
console.log(arrayIterator.next()); // { value: 3, done: false }
console.log(arrayIterator.next()); // { value: undefined, done: true }

for (const value of array) {
  console.log(value);
}
// 1
// 2
// 3

{ value: 1, done: false }
{ value: 2, done: false }
{ value: 3, done: false }
{ value: undefined, done: true }
1
2
3


In [ ]:

const set = new Set([1, 2, 3]);
const setIterator = set[Symbol.iterator]();

console.log(setIterator.next()); // { value: 1, done: false }
console.log(setIterator.next()); // { value: 2, done: false }
console.log(setIterator.next()); // { value: 3, done: false }
console.log(setIterator.next()); // { value: undefined, done: true }

for (const value of set) {
  console.log(value);
}
// 1
// 2
// 3

In [ ]:

const map = new Map([['a', 1], ['b', 2], ['c', 3]]);
const mapIterator = map[Symbol.iterator]();

console.log(mapIterator.next()); // { value: ['a', 1], done: false }
console.log(mapIterator.next()); // { value: ['b', 2], done: false }
console.log(mapIterator.next()); // { value: ['c', 3], done: false }
console.log(mapIterator.next()); // { value: undefined, done: true }

for (const [key, value] of map) {
  console.log(`${key}: ${value}`);
}
// a: 1
// b: 2
// c: 3

const mapEntries = map.entries();

console.log(mapEntries.next()); // { value: ['a', 1], done: false }
console.log(mapEntries.next()); // { value: ['b', 2], done: false }
console.log(mapEntries.next()); // { value: ['c', 3], done: false }
console.log(mapEntries.next()); // { value: undefined, done: true }

for (const entry of map.entries()) {
  console.log(entry);
}
// ['a', 1]
// ['b', 2]
// ['c', 3]

const mapValues = map.values();

console.log(mapValues.next()); // { value: 1, done: false }

for (const value of mapValues) {
  console.log(value);
}
// 2
// 3

const mapKeys = map.keys();

console.log(mapKeys.next()); // { value: 'a', done: false }

for (const key of mapKeys) {
  console.log(key);
}
// b
// c


{ value: [ "a", 1 ], done: false }
{ value: [ "b", 2 ], done: false }
{ value: [ "c", 3 ], done: false }
{ value: undefined, done: true }
a: 1
b: 2
c: 3
{ value: [ "a", 1 ], done: false }
{ value: [ "b", 2 ], done: false }
{ value: [ "c", 3 ], done: false }
{ value: undefined, done: true }
[ "a", 1 ]
[ "b", 2 ]
[ "c", 3 ]
{ value: 1, done: false }
2
3
{ value: "a", done: false }
b
c


: 

### 1.3.2 언어와 이터러블의 상호작용

#### 전개 연산자와 이터러블

배열이나 객체를 복사하거나 및 병합

In [2]:
const array = [1, 2, 3];
const array2 = [...array, 4, 5, 6];

console.log(array2); // [1, 2, 3, 4, 5, 6]

[ 1, 2, 3, 4, 5, 6 ]


이터러블 객체를 배열로 변환

In [3]:
const set = new Set([1, 2, 3]);
const array = [...set];

console.log(array); // [1, 2, 3]

[ 1, 2, 3 ]


함수 호출 시 이터러블 객체의 요소들을 개별 인자로 전달

In [ ]:
const numbers = [1, 2, 3];

function sum(...nums: number[]): number {
  return nums.reduce((a, b) => a + b, 0);
}

console.log(sum(...numbers)); // 6

6


: 

#### 구조 분해 할당과 이터러블

특정 요소 추출

In [ ]:
const array = [1, 2, 3, 4];
const [first, second] = array;

console.log(first);   // 1
console.log(second);  // 2

const [head, ...tail] = array;

console.log(head); // 1
console.log(tail); // [2, 3, 4]

Map 객체의 키-값 쌍을 분해 할당

In [ ]:
const map = new Map();
map.set('a', 1);
map.set('b', 2);
map.set('c', 3);
for (const [key, value] of map.entries()) {
  console.log(`${key}: ${value}`);
}
// a: 1
// b: 2
// c: 3

a: 1
b: 2
c: 3


: 

#### 사용자 정의 이터러블과 전개 연산자

In [ ]:
function naturals(end = Infinity): Iterator<number> {
  let n = 1;
  return {
    next(): IteratorResult<number> {
      return n <= end
        ? { value: n++, done: false }
        : { value: undefined, done: true };
    },
    [Symbol.iterator]() {
      return this;
    }
  }
}

const array = [0, ...naturals(3)];
console.log(array); // [0, 1, 2, 3];

[ 0, 1, 2, 3 ]


: 

### 1.3.3 제너레이터로 만든 이터레이터도 이터러블

#### 제너레이터로 만든 map 함수

In [ ]:
function* map<A, B>(
  f: (value: A) => B, iterable: Iterable<A>
): IterableIterator<B> {
  for (const value of iterable) {
    yield f(value);
  }
}

const array = [1, 2, 3, 4];
const mapped: IterableIterator<number> = map(x => x * 2, array);
const iterator = mapped[Symbol.iterator]();

console.log(mapped.next().value); // 2
console.log(iterator.next().value); // 4
console.log(([...iterator])); // [6, 8]



2
4
[ 6, 8 ]


ReferenceError: naturals is not defined

#### 제너레이터로 만든 이터레이터와 for...of문

In [ ]:
function* map<A, B>(
  f: (value: A) => B, iterable: Iterable<A>
): IterableIterator<B> {
  for (const value of iterable) {
    yield f(value);
  }
}

function naturals(end = Infinity): Iterator<number> {
  let n = 1;
  return {
    next(): IteratorResult<number> {
      return n <= end
        ? { value: n++, done: false }
        : { value: undefined, done: true };
    },
    [Symbol.iterator]() {
      return this;
    }
  }
}

let acc = 0;
for (const num of map((x) => x * 2, naturals(4))) {
  acc += num;
}
console.log(acc); // 20

20


: 

위 코드에서 map 함수에 배열이 아닌 지연 평가되는 이터레이터를 전달하여 실행 과정에서 배열을 만들지 않고도 acc에 모든 값을 더할 수 있다.

## 1.4 이터러블을 다루는 함수형 프로그래밍

### 1.4.1. forEach 함수

In [5]:
function forEach(f, iterable) {
  for (const value of iterable) {
    f(value);
  }
}

const array = [1, 2, 3];
forEach(console.log, array);
// 1
// 2
// 3

1
2
3
